In [1]:
import pyam
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

Step 1: Let us read in the necessary data.

In [3]:
df = pyam.IamDataFrame(
    Path(
        '../data/101_data.xlsx'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/101_data.xlsx
pyam.core - INFO: Reading meta indicators


<IPython.core.display.Javascript object>

In [4]:
df_cdr = (
    df
    .filter(
        variable='*Carbon Removal*'
    )
    .convert_unit(
        current='Gt CO2/yr',
        to='Gt CO2/yr',
        factor=-1
    )
)

<IPython.core.display.Javascript object>

In [5]:
df_compiled = pyam.concat(
    [
        df_cdr,
        df.filter(
            variable='*Carbon Removal*',
            keep=False
        )
    ]
)

<IPython.core.display.Javascript object>

Step 2: We want to identify the regional patterns in contributions to the regional net GHG reductions:
* 2020 - 2030
* 2030 - 2050
* 2050 - 2100

In [6]:
time_periods = [
    (2020, 2030),
    (2030, 2050),
    (2050, 2100)
]

<IPython.core.display.Javascript object>

In [7]:
def format_output_data_frame(df, cols_select,  variable_name, summarise=True):
    """Function to help format the output dataframe"""
    cols_to_melt = set(cols_select) - set('Category')
    melted_data = pd.melt(
        frame=df,
        id_vars=['model', 'scenario', 'Category'],
        value_vars=cols_to_melt
    )
    #Now, we split the variable column
    melted_data.loc[:, 'region'] = (
        melted_data.loc[:,'variable'].apply(
            lambda x: x.split('_')[1]
        )
    )
    melted_data.loc[:, 'timeframe'] = (
        melted_data.loc[:,'variable'].apply(
            lambda x: x.split('_')[2] + '-' + x.split('_')[3]
        )
    )
    melted_data.loc[:, 'variable'] = variable_name
    if summarise:
        summarised_data_raw = (
            melted_data
            .groupby(['Category', 'region', 'timeframe'])['value']
            .describe()
            .loc[:, ['25%', '50%', '75%']]
        )
        summarised_data_raw.loc[:, 'summary'] = (
            summarised_data_raw.apply(
                lambda x: f"{x['50%']}% ({x['25%']}-{x['75%']})",
                axis=1
            )
            
        )
        summarised_data_clean = summarised_data_raw.loc[:, 'summary'].unstack(level=-1)
        summarised_data_clean.loc[:, 'variable'] = variable_name
    return melted_data, summarised_data_clean

<IPython.core.display.Javascript object>

Step 3: Now, for each region, we want to take a closer look at the role of gross reductions, non CO2 and CDR.

In [8]:
def crunch_through_and_summarise_per_region(df, region):
    """Helper function to crunch through the regional summaries"""
    total_variable = 'AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only'
    component_variables = [
        ('AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross', 'gross'),
        ('AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only', 'nonco2'),
        ('AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total', 'cdr')    
    ]
    # Step 1: Pull out the regional data
    regional_data = (
        df
        .filter(
            region=region
        )
    )
    # Step 2
    for start, end in time_periods:
        # First calculate the total mitigation
        df_net = (
            regional_data
            .filter(year=range(start, end+1))
            .offset(year=start)
        )
        # Calculate the total mitigation burden
        regional_data.set_meta(
            meta=(
                df_net
                .filter(variable=total_variable, year=end)
                .timeseries()
                .droplevel(['region', 'variable', 'unit'])[end]
            ),
            name=f'total_mitigation_{start}_{end}'
        )
        # Per variable calculate the contributions to these reductions
        for variable, label in component_variables:
            ts = (
                df_net
                .filter(variable=variable)
                .timeseries()
                .apply(
                    lambda x: x * 100 / regional_data.meta.loc[x.name[0:2], f'total_mitigation_{start}_{end}'],
                    axis=1
                )
                .round(0)
            )
            regional_data.set_meta(
                meta=ts.droplevel(['region', 'variable', 'unit'])[end],
                name=f'{label}_{region}_{start}_{end}'
            )
    # Step 3
    cols = [x[1] for x in component_variables]
    df = []
    df_sum = []
    for var, label in component_variables:
        cols = [x for x in regional_data.meta if f'{label}_{region}' in x]
        cols.append('Category')
        out, out_sum = (
            format_output_data_frame(
                regional_data.meta.reset_index(),
                cols_select=cols,
                variable_name=var
            )
        )
        df.append(out)
        df_sum.append(out_sum)
    
    compiled = pd.concat(df)
    summary = (
        pd.concat(df_sum)
        .set_index('variable', append=True)
        .sort_index(level=0)
    )
    return compiled, summary

<IPython.core.display.Javascript object>

In [9]:
regs = [x for x in df_compiled.region if 'World' not in x]

<IPython.core.display.Javascript object>

In [10]:
regs

['R5ASIA', 'R5LAM', 'R5MAF', 'R5OECD90+EU', 'R5REF']

<IPython.core.display.Javascript object>

In [11]:
final_reg_summary = []
for r in regs:
    _, sum = crunch_through_and_summarise_per_region(
        df_compiled,
        r
    )
    final_reg_summary.append(sum)

<IPython.core.display.Javascript object>

In [12]:
final_reg_summary_pd = pd.concat(final_reg_summary)

<IPython.core.display.Javascript object>

In [13]:
final_reg_summary_pd.to_excel(
    Path(
        'figures/SI_table2.xlsx'
    )
)

<IPython.core.display.Javascript object>

Statements: Patterns for the R5ASIA region.

For the Asia region, we note a dominant contribution (median of around 80%) from gross CO2 emission reductions across all three time periods in the C1 pathways with the balance largely coming from non-CO2 reductions followed by CDR. However, in C2 and C3 pathways where there is slower global near-term action, CDR plays a larger role beyond 2050, accounting for over 20% of the regional net GHG reductions.

In [14]:
final_reg_summary_pd.loc[
    pd.IndexSlice['C1', 'R5ASIA', 'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross'],
    :
]

timeframe
2020-2030    78.0% (66.75-82.0)
2030-2050    79.0% (71.0-85.25)
2050-2100    80.0% (60.0-123.5)
Name: (C1, R5ASIA, AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross), dtype: object

<IPython.core.display.Javascript object>

In [15]:
final_reg_summary_pd.loc[
    pd.IndexSlice[:, 'R5ASIA', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'],
    :
]

,,timeframe,2020-2030,2030-2050,2050-2100
Category,region,variable,,,
C1,R5ASIA,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,7.0% (3.0-17.0),12.5% (4.0-20.0),-5.5% (-30.0-18.5)
C2,R5ASIA,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,4.0% (2.0-11.0),9.0% (4.0-18.25),27.0% (10.75-32.0)
C3,R5ASIA,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,4.0% (-0.0-15.0),7.0% (4.0-12.0),21.0% (11.0-42.0)


<IPython.core.display.Javascript object>

Statement: Patterns for OECD+EU

In [16]:
final_reg_summary_pd.loc[
    pd.IndexSlice[:, 'R5OECD90+EU', :],
    :
].sort_index(level=-1)

,,timeframe,2020-2030,2030-2050,2050-2100
Category,region,variable,,,
C1,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,3.0% (2.0-6.0),21.5% (13.0-33.0),42.5% (-28.25-73.75)
C2,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,2.0% (-0.0-4.0),15.0% (8.75-25.5),48.0% (34.0-66.0)
C3,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total,2.0% (1.0-4.0),11.0% (7.0-24.5),35.0% (24.0-66.0)
C1,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross,76.5% (73.0-82.0),71.0% (62.75-77.0),52.0% (22.0-105.0)
C2,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross,78.0% (75.0-84.0),73.0% (67.0-79.0),47.0% (30.75-60.0)
C3,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Gross,74.0% (68.0-78.0),74.0% (66.0-80.0),58.0% (29.0-69.0)
C1,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only,17.5% (14.0-23.0),7.0% (5.0-14.0),8.0% (2.0-19.25)
C2,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only,18.0% (15.0-21.0),11.0% (9.0-14.0),6.0% (2.0-8.0)
C3,R5OECD90+EU,AR6 Reanalysis|OSCARv3.2|Emissions|NonCO2|Direct Only,23.0% (19.0-28.5),10.0% (7.0-15.0),5.0% (2.0-8.0)


<IPython.core.display.Javascript object>

Statement: Patterns for Latin America

In [17]:
final_reg_summary_pd.loc[
    pd.IndexSlice['C1', 'R5LAM', 'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total'],
    :
]

timeframe
2020-2030      18.0% (14.0-28.5)
2030-2050      32.0% (22.0-42.0)
2050-2100    79.5% (48.75-136.0)
Name: (C1, R5LAM, AR6 Reanalysis|OSCARv3.2|Carbon Removal|Total), dtype: object

<IPython.core.display.Javascript object>